In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from pymongo import MongoClient
from pymongo import MongoClient, GEOSPHERE

from src.BuscarGeooficias import *
from src.buscarOficinas import *
from src.SacarDireccion import *
from src.SacarGeocode import *
from src.MasivoGeocoe import *


import pandas as pd
import requests
import os
import folium
import json
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
client = MongoClient("mongodb://localhost/datamad0320")
db = client.get_database()

In [3]:
!mongoimport --db=datamad0320 --collection=ejercicios --drop ./Output/companies.json


2020-06-03T16:07:14.508+0200	connected to: mongodb://localhost/
2020-06-03T16:07:14.511+0200	dropping: datamad0320.ejercicios
2020-06-03T16:07:16.918+0200	18801 document(s) imported successfully. 0 document(s) failed to import.


In [4]:
all_offices = list(db.ejercicios.find({},{"offices":1,"name":1,"founded_year":1,"category_code":1}))


In [5]:
companias = pd.DataFrame (all_offices )
companias = companias.explode("offices")
companias.head()


,_id,name,category_code,founded_year,offices
0,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,1996.0,"{'description': 'Headquarters', 'address1': '4..."
1,52cdef7c4bab8bd675297d8c,Zoho,software,2005.0,"{'description': 'Headquarters', 'address1': '4..."
2,52cdef7c4bab8bd675297d91,Geni,web,2006.0,"{'description': 'Headquarters', 'address1': '9..."
3,52cdef7c4bab8bd675297d92,Flektor,games_video,NaN,"{'description': None, 'address1': '8536 Nation..."
4,52cdef7c4bab8bd675297d8d,Digg,news,2004.0,"{'description': None, 'address1': '135 Mississ..."


In [6]:
limpias_offices = companias.apply(buscaroficinaGeoPoint,axis=1, result_type="expand")
limpias_offices.columns = ["office","clean_state"]
limpias_offices.head()

,office,clean_state
0,"{'type': 'Point', 'coordinates': [-121.904945,...",success
1,"{'type': 'Point', 'coordinates': [-121.904945,...",success
2,"{'type': 'Point', 'coordinates': [-118.393064,...",success
3,"{'type': 'Point', 'coordinates': [-118.379768,...",success
4,"{'type': 'Point', 'coordinates': [-122.394523,...",success


In [7]:
companias_limpias= pd.concat([companias,limpias_offices], axis=1)

In [8]:
companias_limpias = companias_limpias[["name","category_code","office","clean_state","founded_year"]]

In [9]:
companias_limpias.clean_state.value_counts()

success                     10834
Invalid lat lat and long     5871
No office                    5057
Name: clean_state, dtype: int64

In [10]:
companias_limpias = companias_limpias.drop(companias_limpias.index[companias_limpias["clean_state"]=="No office",])

In [11]:
companias_limpias.clean_state.value_counts()
companias_limpias.head()

,name,category_code,office,clean_state,founded_year
0,AdventNet,enterprise,"{'type': 'Point', 'coordinates': [-121.904945,...",success,1996.0
1,Zoho,software,"{'type': 'Point', 'coordinates': [-121.904945,...",success,2005.0
2,Geni,web,"{'type': 'Point', 'coordinates': [-118.393064,...",success,2006.0
3,Flektor,games_video,"{'type': 'Point', 'coordinates': [-118.379768,...",success,NaN
4,Digg,news,"{'type': 'Point', 'coordinates': [-122.394523,...",success,2004.0


In [12]:
companias_limpias.rename(columns={'office':'coordenadas'}, inplace=True)

In [13]:
companias_limpias = companias_limpias.reset_index()

In [14]:
companias_limpias = companias_limpias.drop(['index'], axis=1)

In [15]:
companias_limpias.head()

,name,category_code,coordenadas,clean_state,founded_year
0,AdventNet,enterprise,"{'type': 'Point', 'coordinates': [-121.904945,...",success,1996.0
1,Zoho,software,"{'type': 'Point', 'coordinates': [-121.904945,...",success,2005.0
2,Geni,web,"{'type': 'Point', 'coordinates': [-118.393064,...",success,2006.0
3,Flektor,games_video,"{'type': 'Point', 'coordinates': [-118.379768,...",success,NaN
4,Digg,news,"{'type': 'Point', 'coordinates': [-122.394523,...",success,2004.0


In [16]:
companias_limpias = companias_limpias.dropna(subset=["founded_year"])
companias_limpias.head()

,name,category_code,coordenadas,clean_state,founded_year
0,AdventNet,enterprise,"{'type': 'Point', 'coordinates': [-121.904945,...",success,1996.0
1,Zoho,software,"{'type': 'Point', 'coordinates': [-121.904945,...",success,2005.0
2,Geni,web,"{'type': 'Point', 'coordinates': [-118.393064,...",success,2006.0
4,Digg,news,"{'type': 'Point', 'coordinates': [-122.394523,...",success,2004.0
5,Fox Interactive Media,web,"{'type': 'Point', 'coordinates': [-118.39417, ...",success,1979.0


In [17]:
companias_limpias.to_json("Input/Companias.json",orient="records")

In [18]:
!mongoimport --db=datamad0320 --collection=CompaniasLimpias --drop --jsonArray ./Input/Companias.json

2020-06-03T16:07:20.129+0200	connected to: mongodb://localhost/
2020-06-03T16:07:20.129+0200	dropping: datamad0320.CompaniasLimpias
2020-06-03T16:07:20.368+0200	14057 document(s) imported successfully. 0 document(s) failed to import.


In [19]:
db.CompaniasLimpias.create_index([("coordenadas", GEOSPHERE)])


'coordenadas_2dsphere'

In [20]:
query = getBuscar("RETIRO Madrid",4000)
query

<Response [200]>


{'Lugares': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [-3.67744, 40.40816]},
   '$maxDistance': 4000}}}

In [21]:
cur=(list(db.CompaniasLimpias.find({"coordenadas": {"$near":{"$geometry":{"type":"Point","coordinates":[ -3.67744,40.40816]},"$maxDistance":5000}}})))

#cur = db.NewYorkCompanis.find(query.update({"clean_state":"success"}), {"_id":0})
#cur =( db.CompaniasLimpias.find({"Lugares":['$near': {'$geometry': {'type': 'Point', 'coordinates':
                #[-3.67744, 40.40816]},'$maxDistance': 5000},{"_id":0}]})

result = cur
len(list(result))

30

In [22]:
madridcompanies = pd.DataFrame(result)
madridcompanies.head(3)

,_id,name,category_code,coordenadas,clean_state,founded_year
0,5ed7ae9849f7b6955370190c,Digital Assets Deployment,ecommerce,"{'type': 'Point', 'coordinates': [-3.678121, 4...",success,2006.0
1,5ed7ae9849f7b69553702743,Busuu,social,"{'type': 'Point', 'coordinates': [-3.6830316, ...",success,2008.0
2,5ed7ae9849f7b695537031df,Tuenti Technologies,social,"{'type': 'Point', 'coordinates': [-3.696277, 4...",success,2006.0


In [23]:
madridcompanies.drop(columns=["_id"],inplace=True)
madridcompanies

,name,category_code,coordenadas,clean_state,founded_year
0,Digital Assets Deployment,ecommerce,"{'type': 'Point', 'coordinates': [-3.678121, 4...",success,2006.0
1,Busuu,social,"{'type': 'Point', 'coordinates': [-3.6830316, ...",success,2008.0
2,Tuenti Technologies,social,"{'type': 'Point', 'coordinates': [-3.696277, 4...",success,2006.0
3,Tagmore Solutions,mobile,"{'type': 'Point', 'coordinates': [-3.6883074, ...",success,2007.0
4,ASPgems,software,"{'type': 'Point', 'coordinates': [-3.6992667, ...",success,2006.0
5,IMASTE,enterprise,"{'type': 'Point', 'coordinates': [-3.6755166, ...",success,2003.0
6,Daily Flat Rental,ecommerce,"{'type': 'Point', 'coordinates': [-3.703248, 4...",success,2008.0
7,Daily Flat Rental,ecommerce,"{'type': 'Point', 'coordinates': [-3.703248, 4...",success,2008.0
8,Nexway,ecommerce,"{'type': 'Point', 'coordinates': [-3.7033074, ...",success,2002.0
9,DOMODESK,mobile,"{'type': 'Point', 'coordinates': [-3.6510254, ...",success,1998.0


In [24]:
madridcompanies.to_json("Input/madridcompanies.json",orient="records")

In [25]:
!mongoimport --db=datamad0320 --collection=madridcompanies --drop --jsonArray ./Input/madridcompanies.json

2020-06-03T16:07:22.086+0200	connected to: mongodb://localhost/
2020-06-03T16:07:22.086+0200	dropping: datamad0320.madridcompanies
2020-06-03T16:07:22.108+0200	30 document(s) imported successfully. 0 document(s) failed to import.


In [26]:
guarderiasretiro = ExtractDireccion("madrid retiro","guarderias", 2)
aeropuerto = ExtractDireccion("madrid","aeropuerto", 1)
starbucks = ExtractDireccion("madrid retiro","starbucks", 1)

/home/sergio/bootcamp/ProyectoMongo/src/SacarDireccion.py:27: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 27 of the file /home/sergio/bootcamp/ProyectoMongo/src/SacarDireccion.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(data.text)


In [27]:
guarderiasretiroDatos = pd.DataFrame(guarderiasretiro)
aeropuertoDatos = pd.DataFrame(aeropuerto)
starbucksDatos = pd.DataFrame(starbucks)

In [28]:
#Renombro las columnas con name y adress
aeropuertoDatos = aeropuertoDatos.rename(columns={0:'name',1:'adress'})
starbucksDatos= starbucksDatos.rename(columns={0:'name',1:'adress'})
guarderiasretiroDatos = guarderiasretiroDatos.rename(columns={0:'name',1:'adress'})

In [29]:
display(guarderiasretiroDatos.head(2),starbucksDatos.head(2),aeropuertoDatos.head(2))

,name,adress
0,Jaizkibel,"Calle de José Del Hierro, 29"
1,Escuela Infantil Pipo’s,"Calle de Cáceres, 50"


,name,adress
0,Starbucks,"Calle de José Ortega y Gasset, 29"
1,Starbucks,"Calle Doctor Drumen, 4"


,name,adress
0,Aeropuerto de Madrid-Barajas Adolfo Suárez,"Avenida de la Hispanidad, s/n"
1,Taxi Aeropuerto Madrid,"Gran Vía, 30"


In [30]:
aeropuertoDatos["coordenadas"]= geocodeGoogleMasivo(aeropuertoDatos)

In [31]:
guarderiasretiroDatos["coordenadas"] = geocodeGoogleMasivo(guarderiasretiroDatos)
guarderiasretiroDatos

,name,adress,coordenadas
0,Jaizkibel,"Calle de José Del Hierro, 29","{'type': 'Point', 'coordinates': [-3.6455125, ..."
1,Escuela Infantil Pipo’s,"Calle de Cáceres, 50","{'type': 'Point', 'coordinates': [-3.7000098, ..."
2,El Bosque Encantado,"Calle de Antonio López, 246","{'type': 'Point', 'coordinates': [-3.6959297, ..."
3,Guarderia Arco de Colores,Calle Gumersindo Azcarate 32,"{'type': 'Point', 'coordinates': [-3.7097628, ..."
4,Guarderia Infantil Primeros Pasos,"Calle Campo de la Paloma, 12","{'type': 'Point', 'coordinates': [-3.6465462, ..."
5,Estrella Amdal,"Calle Francisco Vitoria, 4","{'type': 'Point', 'coordinates': [-3.674009, 4..."
6,Rocho Carmona Maria del Carmen,"Calle Reyes Magos, 6","{'type': 'Point', 'coordinates': [-3.674691499..."
7,Cei Fantasía,Calle Justa García 12,"{'type': 'Point', 'coordinates': [-3.7095009, ..."
8,Niro,"Calle Andres Torrejon, 18","{'type': 'Point', 'coordinates': [-3.6818551, ..."
9,Teddy Centro Infantil,"Calle Leon GIL de Palacio, 5 - LC 10","{'type': 'Point', 'coordinates': [-3.6790747, ..."


In [32]:
starbucksDatos["coordenadas"] =  geocodeGoogleMasivo(starbucksDatos)
starbucksDatos

,name,adress,coordenadas
0,Starbucks,"Calle de José Ortega y Gasset, 29","{'type': 'Point', 'coordinates': [-3.6830042, ..."
1,Starbucks,"Calle Doctor Drumen, 4","{'type': 'Point', 'coordinates': [-3.6931363, ..."
2,Starbucks,"Calle de la Montera, 8","{'type': 'Point', 'coordinates': [-3.7027745, ..."
3,Starbucks,"Calle del Conde de Peñalver, 10","{'type': 'Point', 'coordinates': [-3.6753894, ..."
4,Starbucks,"Calle Arenal, 14","{'type': 'Point', 'coordinates': [-3.7061836, ..."
5,Starbucks,"Plaza Cánovas del Castillo, 5","{'type': 'Point', 'coordinates': [-3.6950046, ..."
6,Starbucks,"Calle de Ciudad Rodrigo, 5","{'type': 'Point', 'coordinates': [-3.7084449, ..."
7,Starbucks,"Calle Gran Vía, 30","{'type': 'Point', 'coordinates': [-3.702459999..."
8,Starbucks - CERRADO,"Plaza de Callao, 4","{'type': 'Point', 'coordinates': [-3.7059615, ..."
9,Starbucks,"Calle Velázquez, 32","{'type': 'Point', 'coordinates': [-3.6839483, ..."


In [33]:
aeropuertoDatos

,name,adress,coordenadas
0,Aeropuerto de Madrid-Barajas Adolfo Suárez,"Avenida de la Hispanidad, s/n","{'type': 'Point', 'coordinates': [-3.5828701, ..."
1,Taxi Aeropuerto Madrid,"Gran Vía, 30","{'type': 'Point', 'coordinates': [-3.702459999..."
2,Objetos Perdidos Aeropuerto de Madrid Barajas,"Calle Aeropuerto de Barajas, s/n","{'type': 'Point', 'coordinates': [-3.5777281, ..."
3,Iberia Puente Aereo,Aeropuerto de Barajas Terminal 4,"{'type': 'Point', 'coordinates': [-3.5917845, ..."
4,Aeropuerto de Madrid-Torrejon,"Calle Peonias, 2","{'type': 'Point', 'coordinates': [-3.6179092, ..."
5,Aeropuerto de Madrid - Cuatro Vientos,"Carretera Barrio de la Fortuna, s/n","{'type': 'Point', 'coordinates': [-3.7714895, ..."
6,Vip Sala Terminal 2,"Avenida General, 43","{'type': 'Point', 'coordinates': [-3.5798852, ..."
7,Aeropuerto De Madrid-Barajas Adolfo Suárez Ter...,Via Lusitana 27,"{'type': 'Point', 'coordinates': [-3.7315917, ..."


In [34]:
#Elimino las filas que no coinciden con un aeropuerto
aeropuertoDatos.drop(aeropuertoDatos.index[[1,4,7]],inplace=True)

In [35]:
starbucksDatos.to_json("Input/starbucksDatosGOOGLE.json",orient="records")
guarderiasretiroDatos.to_json("Input/guarderiasretiroDatosGOOGLE.json",orient="records")
aeropuertoDatos.to_json("Input/aeropuertoDatosGOOGLE.json",orient="records")

In [36]:
!mongoimport --db=datamad0320 --collection=aeropuertoDatosGOOGLE.json --drop --jsonArray ./Input/aeropuertoDatosGOOGLE.json

2020-06-03T16:07:45.919+0200	connected to: mongodb://localhost/
2020-06-03T16:07:45.919+0200	dropping: datamad0320.aeropuertoDatosGOOGLE.json
2020-06-03T16:07:45.936+0200	5 document(s) imported successfully. 0 document(s) failed to import.


In [37]:
!mongoimport --db=datamad0320 --collection=guarderiasretiroDatosGOOGLE.json --drop --jsonArray ./Input/guarderiasretiroDatosGOOGLE.json

2020-06-03T16:07:46.080+0200	connected to: mongodb://localhost/
2020-06-03T16:07:46.080+0200	dropping: datamad0320.guarderiasretiroDatosGOOGLE.json
2020-06-03T16:07:46.098+0200	20 document(s) imported successfully. 0 document(s) failed to import.


In [38]:
!mongoimport --db=datamad0320 --collection=starbucksDatosGOOGLE.json --drop --jsonArray ./Input/starbucksDatosGOOGLE.json

2020-06-03T16:07:46.232+0200	connected to: mongodb://localhost/
2020-06-03T16:07:46.232+0200	dropping: datamad0320.starbucksDatosGOOGLE.json
2020-06-03T16:07:46.252+0200	10 document(s) imported successfully. 0 document(s) failed to import.


In [39]:
import folium
import folium.plugins
def etiquetar(cordenadas, nombre,color="blue"):    
    m = folium.Map(location=[madridcompanies.coordenadas[0]["coordinates"][1],madridcompanies.coordenadas[0]["coordinates"][0]],zoom_start=12, tiles='OpenStreetMap')
    cluster = folium.plugins.MarkerCluster().add_to(m)
    for names,latlong in zip(nombre,cordenadas):
        folium.Marker(
        location=[latlong["coordinates"][1],latlong["coordinates"][0]],
        popup =names,
        icon=folium.Icon(icon='ok-sign',color=color)
        ).add_to(m)

    return m     

In [40]:
mapamadridcompanies = etiquetar(madridcompanies.coordenadas,madridcompanies.name,"green")
mapastarbucksDatos = etiquetar(starbucksDatos.coordenadas,starbucksDatos.name,"red")
mapaguarderiasretiroDatos = etiquetar(guarderiasretiroDatos.coordenadas,guarderiasretiroDatos.name,"purple")
mapaaeropuertos = etiquetar(aeropuertoDatos.coordenadas,aeropuertoDatos.name,("blue"))
mapaaeropuertos

In [41]:
mapaguarderiasretiroDatos,mapamadridcompanies,mapastarbucksDatos

(<folium.folium.Map at 0x7f509f3c3190>,
 <folium.folium.Map at 0x7f509f4c95b0>)

In [42]:
import folium.plugins

m = folium.Map(location=[0, 0], zoom_start=2)
lats = [37.25715519, -31.72884313, -75.27080391, 50.30897559]
lons = [167.20482296, 23.5310324, 164.47413453, 26.87409742]

cluster = folium.plugins.MarkerCluster().add_to(m)
for i in range(0, 4):
    folium.Marker(location=[lats[i], lons[i]], popup=str(i)).add_to(cluster)

folium.LayerControl(collapsed=False).add_to(m)
m

In [43]:
query_busqueda={ "Lugares": {
         "$near": {
           "$geometry": {'type': 'Point',
    'coordinates': [ 40.75677,-73.98964]},
           "$maxDistance": 10000,
         }
       }}


In [44]:
verrest =db.NewYorkRestaurantes.find(query_busqueda)
restaurantesResult=list(verrest)
restaurantesResult

[]